# Neural Networks

In this exercise you will learn how to implement a feedforward neural network and train it with backpropagation.

In [40]:
import numpy as np
from numpy.random import multivariate_normal
from numpy.random import uniform
from scipy.stats import zscore

We define two helper functions "init_toy_data" and "init_model" to create a simple data set to work on and a 2 layer neural network. 

First, we create toy data with categorical labels by sampling from different multivariate normal distributions for each class. 

In [41]:
def init_toy_data(num_samples,num_features, num_classes, seed=3):
    # num_samples: number of samples *per class*
    # num_features: number of features (excluding bias)
    # num_classes: number of class labels
    # seed: random seed
    np.random.seed(seed)
    X=np.zeros((num_samples*num_classes, num_features))
    y=np.zeros(num_samples*num_classes)
    for c in range(num_classes):
        # initialize multivariate normal distribution for this class:
        # choose a mean for each feature
        means = uniform(low=-10, high=10, size=num_features)
        # choose a variance for each feature
        var = uniform(low=1.0, high=5, size=num_features)
        # for simplicity, all features are uncorrelated (covariance between any two features is 0)
        cov = var * np.eye(num_features)
        # draw samples from normal distribution
        X[c*num_samples:c*num_samples+num_samples,:] = multivariate_normal(means, cov, size=num_samples)
        # set label
        y[c*num_samples:c*num_samples+num_samples] = c
    return X,y


In [42]:
def init_model(input_size,hidden_size,num_classes, seed=3):
    # input size: number of input features
    # hidden_size: number of units in the hidden layer
    # num_classes: number of class labels, i.e., number of output units
    np.random.seed(seed)
    model = {}
    # initialize weight matrices and biases randomly
    model['W1'] = uniform(low=-1, high=1, size=(input_size, hidden_size))
    model['b1'] = uniform(low=-1, high=1, size=hidden_size)
    model['W2'] = uniform(low=-1, high=1, size=(hidden_size, num_classes))
    model['b2'] = uniform(low=-1, high=1, size=num_classes)
    return model

In [43]:
# create toy data
X,y= init_toy_data(2,4,3) # 2 samples per class; 4 features, 3 classes
# Normalize data
X = zscore(X, axis=0)
print('X: ' + str(X))
print('y: ' + str(y))

X: [[ 0.39636145  1.09468144 -0.89360845  0.91815536]
 [ 0.94419323 -0.94027869  1.22268078  1.29597409]
 [-1.41577399  1.15477931 -0.62099631  0.08323307]
 [-1.35264614 -0.13598976 -1.14221784  0.26928935]
 [ 0.9352123   0.38225626  1.419864   -1.51152157]
 [ 0.49265316 -1.55544856  0.01427781 -1.0551303 ]]
y: [0. 0. 1. 1. 2. 2.]


We now initialise our neural net with one hidden layer consisting of $10$ units and and an output layer consisting of $3$ units. Here we expect (any number of) training samples with $4$ features. We do not apply any activation functions yet. The following figure shows a graphical representation of this neuronal net. 
<img src="nn.graphviz.png"  width="30%" height="30%">

Model:
\begin{equation}  
    y_k(X, W^{(1)}, W^{(2)}) = h^{(2)} \left ( \sum^M_{m=0} W^{(2)}_{km} h^{(1)} \left ( \sum^D_{d=0} W^{(1)}_{md} Xd \right ) \right )
\end{equation}



Input units:
\begin{equation}  
    X = \begin{pmatrix}x_1 \\ \vdots \\ x_D
    \end{pmatrix} \in \mathbb{R}^D \quad \text{with} \quad D = 4
\end{equation}


Activations $a_m$:
\begin{equation}  
    a = \begin{pmatrix}a_1 \\ \vdots \\ a_m
    \end{pmatrix} = W^{(1)} x \in \mathbb{R}^M \quad \text{with} \quad M = 10
\end{equation}


Hidden Units $z_m$:
\begin{equation}  
    Z_m = h(a_m)
\end{equation}


Output units $y_k$:
\begin{equation}  
    Y = \begin{pmatrix}y_1 \\ \vdots \\ y_k
    \end{pmatrix} = W^{(2)} z \in \mathbb{R}^k \quad \text{with} \quad k = 3
\end{equation}
-> defines the error


Activation functions: 
\begin{equation}  
    h^{(1)} \quad \text{and} \quad h^{(2)}
\end{equation}


- Forward propagation: compute all $a_j$ and $z_j$
- Backward propagation: compute all derivatves $\frac{\partial E_n}{\partial W_{ji}^{(i)}}$ with loss in respect to each model parameter update weights for each model parameter 

\begin{equation}  
    w_{ji} = w_{ji} - \lambda \frac{\partial E_n}{\partial W_{ji}^{(i)}}
\end{equation}

! We need an activation function -> otherwise the neural network is a linear model !

In [74]:
from IPython.display import Image

Image("Screenshot_1.png", width=800)

In [45]:
# initialize model
model = init_model(input_size=4, hidden_size=10, num_classes=3)

print('model: ' + str(model))
print('model[\'W1\'].shape: ' + str(model['W1'].shape))
print('model[\'W2\'].shape: ' + str(model['W2'].shape))
print('model[\'b1\'].shape: ' + str(model['b1'].shape))
print('model[\'b12\'].shape: ' + str(model['b2'].shape))
print('number of parameters: ' + str((model['W1'].shape[0] * model['W1'].shape[1]) + 
     np.sum(model['W2'].shape[0] * model['W2'].shape[1]) + 
     np.sum(model['b1'].shape[0]) +
     np.sum(model['b2'].shape[0] )))

model: {'W1': array([[ 0.10159581,  0.41629565, -0.41819052,  0.02165521,  0.78589391,
         0.79258618, -0.74882938, -0.58551424, -0.89706559, -0.11838031],
       [-0.94024758, -0.08633355,  0.2982881 , -0.44302543,  0.3525098 ,
         0.18172563, -0.95203624,  0.11770818, -0.48149511, -0.16979761],
       [-0.43294984,  0.38627584, -0.11909256, -0.68626452,  0.08929804,
         0.56062953, -0.38727294, -0.55608423, -0.22405748,  0.8727673 ],
       [ 0.95199084,  0.34476735,  0.80566822,  0.69150174, -0.24401192,
        -0.81556598,  0.30682181,  0.11568152, -0.27687047, -0.54989099]]), 'b1': array([-0.18696017, -0.0621195 , -0.46152884, -0.41641445, -0.0846272 ,
        0.72106783,  0.17250581, -0.43302428, -0.44404499, -0.09075585]), 'W2': array([[-0.58917931, -0.59724258,  0.02807012],
       [-0.82554126, -0.03282894, -0.27564758],
       [ 0.41537324,  0.49349245,  0.38218584],
       [ 0.37836083, -0.25279975,  0.33626961],
       [-0.32030267,  0.14558774, -0.34838568]

<b>Exercise 1</b>: Implement softmax layer.

Implement the softmax function given by 

$softmax(x_i) = \frac{e^{x_i}}{{\sum_{j\in 1...J}e^{x_j}}}$, 

where $J$ is the total number of classes, i.e. the length of  **x** .

Note: Implement the function such that it takes a matrix X of shape (N, J) as input rather than a single instance **x**; N is the number of instances.

In [48]:
def softmax(X):
    # Calculate exponential of each element in X
    exp_X = np.exp(X) #e^xi...
    
    # print(exp_X)
    # Calculate sum of exponentials for samples
    sum_exp_X = np.sum(exp_X, axis=1, keepdims=True)
    
    # Calculate softmax probabilities
    scores = exp_X / sum_exp_X
    print(scores)
    
    return scores
    #the function returns the likelihood, on how accurate the model categorised the data(?)

Check if everything is correct.

In [49]:
x = np.array([[0.1, 0.7],[0.7,0.4]])
exact_softmax = np.array([[ 0.35434369,  0.64565631],
                         [ 0.57444252,  0.42555748]])
sm = softmax(x)
difference = np.sum(np.abs(exact_softmax - sm))
try:
    assert difference < 0.000001   
    print("Testing successful.")
except:
    print("Tests failed.")

[[0.35434369 0.64565631]
 [0.57444252 0.42555748]]
Testing successful.


<b>Exercise 2</b>: Implement the forward propagation algorithm for the model defined above.

The activation function of the hidden neurons is a Rectified Linear Unit $relu(x)=max(0,x)$ (to be applied element-wise to the hidden units)
The activation function of the output layer is a softmax function as (as implemented in Exercise 1).

The function should return both the activation of the hidden units (after having applied the $relu$ activation function) (shape: $(N, num\_hidden)$) and the softmax model output (shape: $(N, num\_classes)$). 

In [63]:
def forward_prop(X,model):
    W1=model['W1']
    b1=model['b1']
    W2=model['W2']
    b2=model['b2']
    
    (N, J) = np.shape(X)
    input_layer_potentials = []
    hidden_layer_potentials = []
    hidden_activations = []
    
    for i in range(N):
        input_layer_potentials.append(np.dot(W1.T, X[i, :]) + b1)
        num_hidden = len(input_layer_potentials[i])
        hidden_activation = np.zeros((N, num_hidden))
        
        for j in range(num_hidden):
            if input_layer_potentials[i][j] > 0:
                hidden_activation[i,j] = input_layer_potentials[i][j]
        hidden_layer_potentials.append((np.dot(W2.T, hidden_activation[i,:]) + b2))
    
    hidden_layer_potentials = np.array(hidden_layer_potentials)
    probs = softmax(hidden_layer_potentials)
    
    return hidden_activations, probs

In [64]:
acts,probs = forward_prop(X, model)
correct_probs = np.array([[0.22836388, 0.51816433, 0.25347179],
                            [0.15853289, 0.33057078, 0.51089632],
                            [0.40710319, 0.41765056, 0.17524624],
                            [0.85151353, 0.03656425, 0.11192222],
                            [0.66016592, 0.19839791, 0.14143618],
                            [0.70362036, 0.08667923, 0.20970041]])

# the difference should be very small.
difference =  np.sum(np.abs(probs - correct_probs))

try:
    assert probs.shape==(X.shape[0],len(set(y)))
    assert difference < 0.00001   
    print("Testing successful.")
except:
    print("Tests failed.")

[[0.22836388 0.51816433 0.25347179]
 [0.15853289 0.33057078 0.51089632]
 [0.40710319 0.41765056 0.17524624]
 [0.85151353 0.03656425 0.11192222]
 [0.66016592 0.19839791 0.14143618]
 [0.70362036 0.08667923 0.20970041]]
Testing successful.


<b>Exercise 3:</b> 

How would you train the above defined neural network? Which loss-function would you use? You do not need to implement this.

<b>Part 2 (Neural Net using Keras)</b>

Instead of implementing the model learning ourselves, we can use the neural network library Keras for Python (https://keras.io/). Keras is an abstraction layer that either builds on top of Theano or Google's Tensorflow. So please install Keras and Tensorflow/Theano for this lab.

<b>Exercise 4:</b>
    Implement the same model as above using Keras:
    
    ** 1 hidden layer à 10 units
    ** softmax output layer à three units
    ** 4 input features
    
Compile the model using categorical cross-entropy (also referred to as 'softmax-loss') as loss function and using categorical crossentropy together with categorical accuracy as metrics for runtime evaluation during training.

Hint 1: Use the Sequential Class API of Keras (https://keras.io/api/models/sequential/ or https://www.tensorflow.org/guide/keras/sequential_model)

Hint 2: You can use the Adam optimizer of Keras for the model compilation

In [66]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers.core import Activation
from keras.optimizers import Adam

# define the model 
model = Sequential()

# 1 hidden layer with 10 units 
model.add(Dense(10, input_dim=4, activation='relu')) 

# output layer
model.add(Dense(3, activation='softmax')) 


# compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])

The description of the current network can always be looked at via the summary method. The layers can be accessed via model.layers and weights can be obtained with the method get_weights. Check if your model is as expected. 

In [67]:
# Check model architecture and initial weights.

W_1, b_1 = model.layers[0].get_weights()
print("First layer weights: %s; shape: %s" % (W_1,W_1.shape))
print("First layer bias: %s; shape: %s" % (b_1,b_1.shape))
W_2, b_2 = model.layers[1].get_weights()
print("Second layer weights: %s; shape: %s" % (W_2,W_2.shape))
print("Second layer bias: %s; shape: %s" % (b_2,b_2.shape))
print("number of layes: " + str(len(model.layers)))
model.summary()


First layer weights: [[-2.64628530e-02 -3.17941487e-01  1.40096009e-01  1.75264359e-01
  -5.55876195e-01 -2.65359879e-04 -2.69887239e-01  3.90473247e-01
   1.78388596e-01  1.17118776e-01]
 [-3.99027288e-01 -1.56297445e-01  5.03492594e-01  4.02462363e-01
   1.13974869e-01 -5.75973570e-01 -1.36821270e-02  6.89988136e-02
  -3.61715674e-01  4.16068077e-01]
 [-6.40582621e-01  1.12818718e-01 -8.02006721e-02  2.31552064e-01
  -4.47050005e-01 -4.09130603e-01  3.68958116e-02 -3.37841570e-01
   9.34362411e-02  1.57646477e-01]
 [ 5.19595027e-01  2.51585245e-01  1.30234659e-01 -2.85279870e-01
  -4.91653591e-01  2.38664865e-01  3.10264647e-01 -5.12333035e-01
   6.41079664e-01  4.93749261e-01]]; shape: (4, 10)
First layer bias: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]; shape: (10,)
Second layer weights: [[-5.87117076e-02  4.29862380e-01 -6.73914552e-01]
 [-1.50469661e-01 -3.84609103e-02 -2.23552316e-01]
 [ 5.59570789e-01 -3.30684364e-01 -4.22358781e-01]
 [ 5.38722396e-01  5.90068579e-01  3.41091275e-01]
 [ 4

<b>Exercise 5:</b> Train the model on the toy data set generated below: 

Hints: 

* Keras expects one-hot-coded labels 

* Don't forget to normalize the data

In [69]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from tensorflow.keras.utils import to_categorical

X, y = init_toy_data(1000,4,3, seed=3)
print(np.unique(np.array(y)))
y_encod = to_categorical(y, num_classes=3)
print(y_encod)

X_train, X_test, y_train, y_test = train_test_split(X,y_encod,test_size=0.33,random_state=67)

scaler = StandardScaler()
scaler.fit(X_train)
X_norm_train = scaler.transform(X_train)
X_norm_test = scaler.transform(X_test)

[0. 1. 2.]
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [72]:
history = model.fit(X_norm_train, y_train, epochs = 10)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_norm_test, y_test, verbose=2)
print(f'Test Loss: {test_loss: 4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

train_loss, train_accuracy = model.evaluate(X_norm_train, y_train, verbose=2)
print(f'Train Loss: {train_loss:.4f}')
print(f'Train Accuracy: {train_accuracy:.4f}')

Epoch 1/10
63/63 [==============================] - 0s 526us/step - loss: 0.0536 - categorical_accuracy: 0.9985
Epoch 2/10
63/63 [==============================] - 0s 419us/step - loss: 0.0442 - categorical_accuracy: 0.9985
Epoch 3/10
63/63 [==============================] - 0s 427us/step - loss: 0.0373 - categorical_accuracy: 0.9985
Epoch 4/10
63/63 [==============================] - 0s 391us/step - loss: 0.0320 - categorical_accuracy: 0.9985
Epoch 5/10
63/63 [==============================] - 0s 418us/step - loss: 0.0279 - categorical_accuracy: 0.9985
Epoch 6/10
63/63 [==============================] - 0s 416us/step - loss: 0.0247 - categorical_accuracy: 0.9985
Epoch 7/10
63/63 [==============================] - 0s 400us/step - loss: 0.0220 - categorical_accuracy: 0.9985
Epoch 8/10
63/63 [==============================] - 0s 400us/step - loss: 0.0199 - categorical_accuracy: 0.9985
Epoch 9/10
63/63 [==============================] - 0s 391us/step - loss: 0.0180 - categorical_accuracy:

Compare the test accuracy with the train accuracy. What can you see? Is the model performing well?